In [1]:
import cv2
import numpy as np
from keras.models import load_model
from PIL import Image
import pickle
import matplotlib.pyplot as plt
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import PIL
import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:


# Load the trained model
model = load_model('/content/drive/MyDrive/FACE RECOGNIZATION SYSTEM_model.keras')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Recompile with explicit metrics
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])

# Check if metrics are now available
print(model.metrics_names)


['loss', 'compile_metrics']


In [4]:
model.evaluate

<bound method TensorFlowTrainer.evaluate of <Sequential name=sequential, built=True>>

In [5]:
model.summary()


# Ensure to print the model input shape for verification
print("Model input shape:", model.input_shape)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 128)    │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     7,372,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,410,947 (28.27 MB)

 Trainable params: 7,410,947 (28.27 MB)

 Non-trainable params: 0 (0.00 B)

Model input shape: (None, 128, 128, 3)


#**Camera** **ON**

In [ ]:
import cv2
import numpy as np
from base64 import b64decode, b64encode
import PIL.Image
import io
import pickle
from IPython.display import display, Javascript

# Load the Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the training data (X and y) and class labels
pickle_in = open("/content/drive/MyDrive/data.pkl", "rb")
data_load = pickle.load(pickle_in)  # Images

pickle_in = open("/content/drive/MyDrive/label_encoder.pkl", "rb")
label_encoder= pickle.load(pickle_in)  # Labels

# Load class labels (names)
with open('/content/drive/MyDrive/FACE RECOGNIZATION SYSTEM/categories.pkl', 'rb') as f:
    class_labels = pickle.load(f)

print("Loaded Class Labels:", class_labels)
print("Loaded Labels:", class_labels)



# Function to preprocess the face for the model
def preprocess_face(face, target_shape):
    face_resized = cv2.resize(face, (target_shape[1], target_shape[2]))
    face_normalized = face_resized / 255.0
    face_array = np.expand_dims(face_normalized, axis=0)
    if len(model.input_shape) == 2:
        face_array = face_array.reshape((1, -1))
    return face_array

# Function to handle predictions
def predict_label(face_array):
    predictions = model.predict(face_array, verbose=0)
    confidence = np.max(predictions)
    label_index = np.argmax(predictions)

    return label_index, confidence


# Function to convert JavaScript object to OpenCV image
def js_to_image(js_reply):
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

# Function to convert OpenCV bounding box to base64 for overlay
def bbox_to_bytes(bbox_array):
    # Ensure bbox_array is in RGBA format
    if len(bbox_array.shape) == 3 and bbox_array.shape[2] != 4:
        bbox_array = cv2.cvtColor(bbox_array, cv2.COLOR_BGR2BGRA)  # Convert to BGRA if not already

    # Convert NumPy array to PIL Image
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')

    # Create an in-memory byte buffer
    iobuf = io.BytesIO()

    # Save the image as PNG to the byte buffer
    bbox_PIL.save(iobuf, format='PNG')  # Use PNG to retain transparency

    # Convert to base64 string and return
    bbox_bytes = 'data:image/png;base64,{}'.format(str(b64encode(iobuf.getvalue()), 'utf-8'))
    return bbox_bytes


# JavaScript for video stream
def video_stream():
    js = Javascript('''
        var video;
        var div = null;
        var stream;
        var captureCanvas;
        var imgElement;
        var labelElement;

        var pendingResolve = null;
        var shutdown = false;

        function removeDom() {
            stream.getVideoTracks()[0].stop();
            video.remove();
            div.remove();
            video = null;
            div = null;
            stream = null;
            imgElement = null;
            captureCanvas = null;
            labelElement = null;
        }

        function onAnimationFrame() {
            if (!shutdown) {
                window.requestAnimationFrame(onAnimationFrame);
            }
            if (pendingResolve) {
                var result = "";
                if (!shutdown) {
                    captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
                    result = captureCanvas.toDataURL('image/jpeg', 0.8);
                }
                var lp = pendingResolve;
                pendingResolve = null;
                lp(result);
            }
        }

        async function createDom() {
            if (div !== null) {
                return stream;
            }

            div = document.createElement('div');
            div.style.border = '2px solid black';
            div.style.padding = '3px';
            div.style.width = '100%';
            div.style.height = '100%';
            div.style.maxWidth = '600px';
            document.body.appendChild(div);

            const modelOut = document.createElement('div');
            modelOut.innerHTML = "Status:";
            labelElement = document.createElement('span');
            labelElement.innerText = 'No data';
            labelElement.style.fontWeight = 'bold';
            modelOut.appendChild(labelElement);
            div.appendChild(modelOut);

            video = document.createElement('video');
            video.style.display = 'block';
            video.width = div.clientWidth - 6;
            video.setAttribute('playsinline', '');
            video.onclick = () => { shutdown = true; };
            stream = await navigator.mediaDevices.getUserMedia(
                {video: { facingMode: "environment"}});
            div.appendChild(video);

            imgElement = document.createElement('img');
            imgElement.style.position = 'absolute';
            imgElement.style.zIndex = 1;
            imgElement.onclick = () => { shutdown = true; };
            div.appendChild(imgElement);

            const instruction = document.createElement('div');
            instruction.innerHTML =
                '' +
                '';
            div.appendChild(instruction);
            instruction.onclick = () => { shutdown = true; };

            // Add key press event listener
            window.addEventListener('keydown', function(e) {
                if (e.key === 's' || e.key === 'S') {
                    shutdown = true;  // Stop the video stream on 'S' key press
                }
            });

            video.srcObject = stream;
            await video.play();

            captureCanvas = document.createElement('canvas');
            captureCanvas.width = 640;
            captureCanvas.height = 480;
            window.requestAnimationFrame(onAnimationFrame);

            return stream;
        }

        async function stream_frame(label, imgData) {
            if (shutdown) {
                removeDom();
                shutdown = false;
                return '';
            }

            var preCreate = Date.now();
            stream = await createDom();

            var preShow = Date.now();
            if (label != "") {
                labelElement.innerHTML = label;
            }

            if (imgData != "") {
                var videoRect = video.getClientRects()[0];
                imgElement.style.top = videoRect.top + "px";
                imgElement.style.left = videoRect.left + "px";
                imgElement.style.width = videoRect.width + "px";
                imgElement.style.height = videoRect.height + "px";
                imgElement.src = imgData;
            }

            var preCapture = Date.now();
            var result = await new Promise(function(resolve, reject) {
                pendingResolve = resolve;
            });
            shutdown = false;

            return {'create': preShow - preCreate,
                    'show': preCapture - preShow,
                    'capture': Date.now() - preCapture,
                    'img': result};
        }
    ''')
    display(js)

def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

# Start video stream
video_stream()
label_html = 'Capturing...'
bbox = ''
count = 0

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # Convert JS response to OpenCV image
    img = js_to_image(js_reply["img"])
    bbox_array = np.zeros([480, 640, 3], dtype=np.uint8)  # Use 3 channels for BGR format

    # Convert image to grayscale for face detection
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Draw bounding box and label on the image
    for (x, y, w, h) in faces:
        # Extract and preprocess the detected face
        face = img[y:y+h, x:x+w]
        face_array = preprocess_face(face, target_shape=model.input_shape)

        # Predict the class and confidence
        label_index, confidence = predict_label(face_array)

        # Determine the label based on confidence
        if confidence > 0.8:  # Lowered confidence threshold for debugging
            label = f"{class_labels[label_index]} ({confidence*100:.2f}%)"
            color = (0, 255, 0)  # Green for recognized
        else:
            label = "Unknown"
            color = (0, 0, 255)  # Red for unknown

        # Draw the bounding box and label on the image
        cv2.rectangle(bbox_array, (x, y), (x+w, y+h), color, 2)
        cv2.putText(bbox_array, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)


    # Convert bbox_array to RGB format if not already
    bbox_array = cv2.cvtColor(bbox_array, cv2.COLOR_BGR2RGB)

    # Create an alpha channel for transparency
    # Reduce the alpha value for partial transparency (e.g., 128 for 50% transparency)
    alpha_channel = np.full(bbox_array.shape[:2], 128, dtype=np.uint8)

    # Add the alpha channel to the bbox_array
    bbox_array_with_alpha = np.dstack([bbox_array, alpha_channel])

    # Convert the RGBA array to bytes
    bbox_bytes = bbox_to_bytes(bbox_array_with_alpha)

    # Send the overlay with the bounding box back to the frontend
    bbox = bbox_bytes



Loaded Class Labels: {0: np.str_('AVISHEK'), 1: np.str_('LAMYEA'), 2: np.str_('SAGOR')}
Loaded Labels: {0: np.str_('AVISHEK'), 1: np.str_('LAMYEA'), 2: np.str_('SAGOR')}


<IPython.core.display.Javascript object>

/tmp/ipython-input-1622287508.py:60: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
